### Test for websockets_manager

In [16]:
from decimal import Decimal

import asyncio
import json
import logging
import websockets
# from custom_types import PriceLevel
from typing import List, Dict, Any, Optional
from websockets_manager import WebsocketManager
from orderbook import OrderBook

logger = logging.getLogger(__name__)

import os
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
API_KEY = os.getenv("CLOB_API_KEY")
API_SECRET = os.getenv("CLOB_SECRET")
API_PASSPHRASE = os.getenv("CLOB_PASS_PHRASE")
AGENCY_ADDRESS = os.getenv("AGENCY_ADDRESS")

YES_ID = "46171502957028595838694604499378583099478369877679255114043087599679007467964"
NO_ID = "21457067342029925682172110629429793886827168158271370485467219837949936707498"
CONDITION_ID = "0x07ab34a38692464baf22c7e859550712bb2093fe005b24c58a0717f6e9595e55"

MARKET_WSS_ENDPOINT = "wss://ws-subscriptions-clob.polymarket.com/ws/market"
USER_WSS_ENDPOINT = "wss://ws-subscriptions-clob.polymarket.com/ws/user"

In [ ]:
import os
from py_clob_client.constants import POLYGON
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs
from py_clob_client.order_builder.constants import BUY

host = "https://clob.polymarket.com"
key = os.getenv("PK")
chain_id = POLYGON
print(key)

# Create CLOB client and get/set API credentials
client = ClobClient(host, key=key, chain_id=chain_id)


In [18]:
book = OrderBook(CONDITION_ID, YES_ID, NO_ID)

In [13]:
# the closure
def create_market_handler(order_book: OrderBook, yes_id: str = YES_ID):
    async def handler(data):
        for item in data:
            if item.get('asset_id') == yes_id:
                if item.get('event_type') == 'book':
                    order_book.update_book(item.get('bids'), item.get('asks'))
                elif item.get('event_type') == 'price_change':
                    order_book.handle_price_change(item.get('changes'))
    return handler

    # print(f"Market data received: {data}")

async def example_user_handler(data):
    print(f"User data received: {data}")

In [ ]:
async def create_user_handler(order_book: OrderBook, my_key: str, yes_id: str = YES_ID, no_id: str = NO_ID):
    # my_key: api_key
    async def handler(data):
        for item in data:
            if item.get('event_type') == 'trade' and item.get('asset_id') in {yes_id, no_id} and item.get('status') == 'MATCHED':
                taker_asset_id = item.get('asset_id')
                side = item.get('side')
                
                # Iterate over maker_orders
                for maker_order in item.get('maker_orders', []):
                    if maker_order.get('owner') == my_key:  # Check if I am the maker, key
                        size = maker_order.get('matched_amount', 0)

                        if maker_order.get('asset_id') == taker_asset_id:  # same asset id
                            if taker_asset_id == yes_id:  # both YES token
                                if side == 'BUY': # taker buyer
                                    order_book.yes_position -= Decimal(size)
                                else: 
                                    order_book.yes_position += Decimal(size)
                            else: 
                                if side == 'BUY': 
                                    order_book.no_position -= Decimal(size)
                                else: 
                                    order_book.no_position += Decimal(size)
                        else:  # trade on yes and no token, not same
                            if taker_asset_id == yes_id: 
                                if side == 'BUY':
                                    order_book.no_position += Decimal(size)
                                else: 
                                    order_book.no_position -= Decimal(size)
                            else:  # taker on no token & maker on yes token
                                if side == 'BUY':  
                                    order_book.yes_position += Decimal(size)
                                else: 
                                    order_book.yes_position -= Decimal(size)
                
                # Check if I am the taker
                if item.get('owner') == my_key:
                    size = item.get('size')
                    if taker_asset_id == yes_id:
                        if side == 'BUY':
                            order_book.yes_position += Decimal(size)
                        elif side == 'SELL':
                            order_book.yes_position -= Decimal(size)

                    elif taker_asset_id == no_id:
                        if side == 'BUY':
                            order_book.no_position += Decimal(size)
                        elif side == 'SELL':
                            order_book.no_position -= Decimal(size)

    return handler


In [14]:
manager = WebsocketManager(
    market_url=MARKET_WSS_ENDPOINT,
    user_url=USER_WSS_ENDPOINT,
    api_key=API_KEY,
    api_secret=API_SECRET,
    api_passphrase=API_PASSPHRASE,
    condition_id=CONDITION_ID,
    yes_id=YES_ID,
    no_id=NO_ID
)

manager.add_market_handler(create_market_handler(book))
manager.add_user_handler(example_user_handler)

In [15]:
try:
    await manager.start()
except KeyboardInterrupt:
    # logger.info("Shutting down...")
    await manager.stop()

Sent market subscription: {'assets_ids': ['46171502957028595838694604499378583099478369877679255114043087599679007467964', '21457067342029925682172110629429793886827168158271370485467219837949936707498'], 'type': 'market'}
Sent user subscription: {'auth': {'apiKey': '689c3290-d89e-a3ba-4548-c21125c1efab', 'secret': 'HLmOg9rCRe046UMqgHmTPHfGyz9GVSoFRDw9w4E1Y0U=', 'passphrase': 'e28daa72920b234eeb1f9b6bf19cb62318f96dea2545bd2ec56f4985d788d853'}, 'markets': ['0x07ab34a38692464baf22c7e859550712bb2093fe005b24c58a0717f6e9595e55'], 'type': 'user'}


CancelledError: 